In [5]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [22]:
import time
import random
import numpy as np
from numba import jit
import tensorflow as tf

def site_percolation(n, p, megjelenit=False): # Ez egy kicsit gyorsabb
  with tf.device('/device:GPU:0'):
    # create an n x n grid of sites
    sites = [[False for _ in range(n)] for _ in range(n)]
    # open each site with probability p
    for i in range(n):
        for j in range(n):
            if random.random() < p:
                sites[i][j] = True
    if megjelenit:
      table=sites.copy()
    # check if there is a path of open sites from top to bottom
    visited = set()
    stack = [(0, j) for j in range(n) if sites[0][j]]

    while stack:
        i, j = stack.pop()

        if i == n - 1:
            return True

        visited.add((i, j))

        for di, dj in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            ni, nj = i + di, j + dj

            if 0 <= ni < n and 0 <= nj < n and sites[ni][nj] and (ni, nj) not in visited:
                stack.append((ni, nj))
    if megjelenit:
      return 
    return False

def estimate_pc(n, trials, p_precision):    # Ez egy kicsit gyorsabb
    with tf.device('/device:GPU:0'):
      answer = []
      a = 0.58
      legjobb_e = 1
      legjobb = 1
      for p in [a+i / (40*p_precision) for i in range(1, p_precision)]:
          successes = 0
          for _ in range(trials):
              #board=np.random.rand(n,n)
              if site_percolation(n,p):     #board, p):
                  successes += 1

          estimate = successes / trials
          if int((p-a)*p_precision*40)%100 == 0:
            print(p)

          if abs(estimate - 0.5) < legjobb_e:
            legjobb = p
            legjobb_e = abs(estimate - 0.5)
          if abs(estimate - 0.5) < (0.1/trials):
              answer.append(p)
      return answer, legjobb

def Gpu():
  with tf.device('/device:GPU:0'):
    start_time = time.perf_counter()
    # Calculate elapsed time
    # example usage
    n = 100
    trials = 1000
    p_precision = 1000
    pc = estimate_pc(n, trials, p_precision)

    print(f"Várt kritikus valószínűség az {n}x{n}-es négyzetrácson: {pc}")
    end_time = time.perf_counter()
    print("Elapsed time: ", end_time - start_time)
    return pc

In [19]:
pc=Gpu()

0.5801
0.59
0.5901
0.6
Várt kritikus valószínűség az 100x100-es négyzetrácson: ([0.5913999999999999, 0.592, 0.5926, 0.594, 0.5943999999999999], 0.5913999999999999)
Elapsed time:  281.00122495999994


In [20]:
megoldasok = pc[0].copy()
print(megoldasok)

[0.5913999999999999, 0.592, 0.5926, 0.594, 0.5943999999999999]


In [21]:
egy_megoldas = megoldasok.copy()
print(egy_megoldas)

[0.5913999999999999, 0.592, 0.5926, 0.594, 0.5943999999999999]


In [ ]:

def site_percolation1(board, p):
    new_arr = np.where(board < p, True, False)

    visited = set()
    stack = [(0, j) for j in range(n) if new_arr[0][j]]

    while stack:
        i, j = stack.pop()

        if i == n - 1:
            return True

        visited.add((i, j))

        for di, dj in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            ni, nj = i + di, j + dj

            if 0 <= ni < n and 0 <= nj < n and new_arr[ni][nj] and (ni, nj) not in visited:
                stack.append((ni, nj))
    return False

def estimate_pc1(n, trials, p_precision):
    answer = []
    a = 0.58
    successes=np.zeros(p_precision)
    probs = [a+i / (40*p_precision) for i in range(p_precision)]
    for _ in range(trials):
        board=np.random.rand(n,n)
        for idx, p in enumerate(probs):
            if site_percolation(board, p):
                successes[idx] += 1
        #print(successes)

    estimate = successes / trials
    #print(estimate," estimate")
    good_idx=(abs(estimate-0.5) < (0.5/trials)).nonzero()
    #print(good_idx)
    for i in good_idx[0]:
      answer.append(probs[i])
    #print(good_idx," good_idx")
    #answer = probs[good_idx]
    return answer
